In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import tabula
import pandas as pd
from pathlib import Path
import hashlib
import random
from collections import defaultdict

def get_template_signature(pdf_path):
    try:
        tables = tabula.read_pdf(pdf_path, pages=1, multiple_tables=True)
        if not tables:
            return None

        df = tables[0]
        header_line = "|".join(str(c).strip().lower() for c in df.columns)
        header_hash = hashlib.md5(header_line.encode()).hexdigest()
        return {
            "file": pdf_path.name,
            "headers": list(df.columns),
            "hash": header_hash
        }
    except Exception as e:
        print(f"Error reading {pdf_path.name}: {e}")
        return None


root = Path("../data/raw")
year_folders = [f for f in root.iterdir() if f.is_dir()]

signatures = []

for year_folder in year_folders:
    pdfs = list(year_folder.glob("*.pdf"))
    sample = random.sample(pdfs, min(len(pdfs), 5))  # pick 5 or fewer if not enough
    print(f"\nScanning {len(sample)} PDFs from {year_folder.name}...")
    
    for pdf in sample:
        sig = get_template_signature(pdf)
        if sig:
            signatures.append(sig)

# Group and summarize
by_template = defaultdict(list)
for sig in signatures:
    by_template[sig["hash"]].append(sig["file"])

print(f"\nDetected {len(by_template)} unique table layouts across samples.")
for h, files in by_template.items():
    print("\nTemplate hash:", h)
    print("Example file:", files[0])
    headers = [s['headers'] for s in signatures if s['hash'] == h][0]
    print("Columns:", headers)



Scanning 5 PDFs from 2025...

Scanning 5 PDFs from 2024...

Scanning 5 PDFs from 2023...

Detected 12 unique table layouts across samples.

Template hash: 1de9480f697fb8a9feb721ba3f64359f
Example file: 2025-07-14.pdf
Columns: ['Unnamed: 0', 'Unnamed: 1', 'Rs./kg', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

Template hash: f77f79383807770100ff96ea90031055
Example file: 2025-08-18.pdf
Columns: ['s', 'Rs./kg', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4']

Template hash: f6e6f9ab42ea996b869f70058c692fb7
Example file: 2025-09-22.pdf
Columns: ['Unnamed: 0', 'Rs./kg', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3']

Template hash: 7f3d5432ae71079a291595318cee250a
Example file: 2025-10-16.pdf
Columns: ['Unnamed: 0', 'Rs./kg', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']

Template hash: dd073a0505ac5aba85754bfb1474244c
Example file: 2025-06-11.pdf
Columns: ['s', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnam

In [5]:
# ─────────────────────────────────────────────────────────────
# PDF Layout Profiler (Tabula-based)
# Detects and groups distinct table layouts across PDF samples
# ─────────────────────────────────────────────────────────────

import tabula
import pandas as pd
from pathlib import Path
import hashlib
import random
from collections import defaultdict

# ───────────────────────────────────────────────
# STEP 1 — Extract a "signature" from a single PDF
# ───────────────────────────────────────────────
def get_template_signature(pdf_path):
    """Reads the first table in the first page and returns its column signature."""
    try:
        tables = tabula.read_pdf(pdf_path, pages=1, multiple_tables=True)
        if not tables:
            return None

        df = tables[0]
        header_line = "|".join(str(c).strip().lower() for c in df.columns)
        header_hash = hashlib.md5(header_line.encode()).hexdigest()

        return {
            "file": pdf_path.name,
            "year": pdf_path.parent.name,
            "headers": list(df.columns),
            "hash": header_hash,
        }

    except Exception as e:
        print(f"⚠️ Error reading {pdf_path.name}: {e}")
        return None


# ───────────────────────────────────────────────
# STEP 2 — Sample PDFs from each year folder
# ───────────────────────────────────────────────
root = Path("../data/raw")
year_folders = [f for f in root.iterdir() if f.is_dir()]

signatures = []

for year_folder in year_folders:
    pdfs = list(year_folder.glob("*.pdf"))
    if not pdfs:
        continue

    # Take 5 random samples (or fewer if limited)
    sample = random.sample(pdfs, min(len(pdfs), 5))
    print(f"\n📂 Scanning {len(sample)} PDFs from {year_folder.name}...")

    for pdf in sample:
        sig = get_template_signature(pdf)
        if sig:
            signatures.append(sig)


# ───────────────────────────────────────────────
# STEP 3 — Group by unique header hash
# ───────────────────────────────────────────────
by_template = defaultdict(list)
for sig in signatures:
    by_template[sig["hash"]].append(sig["file"])

print(f"\n🧩 Detected {len(by_template)} unique table layouts across samples.\n")

for h, files in by_template.items():
    example = files[0]
    headers = [s['headers'] for s in signatures if s['hash'] == h][0]
    print("───────────────────────────────")
    print(f"Template hash: {h}")
    print(f"Example file: {example}")
    print("Columns:", headers)


# ───────────────────────────────────────────────
# STEP 4 — Save summary for manual inspection
# ───────────────────────────────────────────────
df_summary = pd.DataFrame(signatures)
df_summary.to_csv("layout_summary.csv", index=False)

print("\n✅ Saved layout_summary.csv — open it to inspect headers and years.")
print("   Use this to identify repeating patterns and name your templates, e.g.:")
print("   TEMPLATE_A (2023 early), TEMPLATE_B (2024 mid), TEMPLATE_C (2025 onward)\n")

# ───────────────────────────────────────────────
# STEP 5 — Optional: define a template map
# (You’ll fill this manually after inspection)
# ───────────────────────────────────────────────
TEMPLATE_MAP = {
    # "hashcode": "template_label",
    # Example:
    # "1de9480f697fb8a9feb721ba3f64359f": "2025_layout_A",
}

def route_parser(pdf_path, header_hash):
    parser = TEMPLATE_MAP.get(header_hash)
    if not parser:
        print(f"⚠️ Unknown layout for {pdf_path.name} (hash={header_hash})")
        return None

    # Later: route to correct parse function
    print(f"→ Parsing {pdf_path.name} using {parser}")
    # return parse_template_x(pdf_path)



📂 Scanning 5 PDFs from 2025...

📂 Scanning 5 PDFs from 2024...

📂 Scanning 5 PDFs from 2023...

🧩 Detected 10 unique table layouts across samples.

───────────────────────────────
Template hash: 7f3d5432ae71079a291595318cee250a
Example file: 2025-10-02.pdf
Columns: ['Unnamed: 0', 'Rs./kg', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5']
───────────────────────────────
Template hash: 4166c890be3a02e4901d5f6d6d82bcaa
Example file: 2025-10-13.pdf
Columns: ['Unnamed: 0', 'Rs./kg', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8']
───────────────────────────────
Template hash: f53962ef4124e803fff59a30ca4f6f97
Example file: 2025-04-07.pdf
Columns: ['s', 'Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
───────────────────────────────
Template hash: be55725588a4d7200672fea800fda6b9
Example file: 2025-08-26.pdf
Columns: ['Unnamed: 0', 'Rs./kg', 'Unn

In [6]:
df_summary.head()

,file,year,headers,hash
0,2025-10-02.pdf,2025,"[Unnamed: 0, Rs./kg, Unnamed: 1, Unnamed: 2, U...",7f3d5432ae71079a291595318cee250a
1,2025-10-13.pdf,2025,"[Unnamed: 0, Rs./kg, Unnamed: 1, Unnamed: 2, U...",4166c890be3a02e4901d5f6d6d82bcaa
2,2025-04-07.pdf,2025,"[s, Unnamed: 0, Unnamed: 1, Unnamed: 2, Unname...",f53962ef4124e803fff59a30ca4f6f97
3,2025-08-26.pdf,2025,"[Unnamed: 0, Rs./kg, Unnamed: 1, Unnamed: 2, U...",be55725588a4d7200672fea800fda6b9
4,2025-01-02.pdf,2025,"[s, Rs./kg, Unnamed: 0, Unnamed: 1, Unnamed: 2...",a9e43d341cf0e5634d23f7eaa613f2e1


Detected 12 columns → ['item', 'unit', 'wholesale_pettah_today', 'wholesale_dambulla_today', 'retail_pettah_today', 'retail_dambulla_today', 'retail_nara_today', 'extra_0', 'extra_1', 'extra_2', 'extra_3', 'extra_4']
      item  unit  wholesale_pettah_today  wholesale_dambulla_today  \
0      NaN  Item                     NaN                       NaN   
1      NaN   NaN                     NaN                       NaN   
2      NaN   NaN                     NaN                       NaN   
3      NaN   NaN                     NaN                       NaN   
4    Beans   NaN                     NaN                     650.0   
5   Carrot   NaN                     NaN                     250.0   
6  Cabbage   NaN                     NaN                     130.0   
7   Tomato   NaN                     NaN                     120.0   
8  Brinjal   NaN                     NaN                     300.0   
9  Pumpkin   NaN                     NaN                      80.0   

   retail_pe

In [26]:
import pdfplumber

with pdfplumber.open("../data/raw/2025/2025-07-30.pdf") as pdf:
    page = pdf.pages[1]
    text = page.extract_text()
    print(text[:1000])  # print first 1000 chars

lines = text.splitlines()
start_idx = next(i for i, l in enumerate(lines) if "RICE" in l)
end_idx = next(i for i, l in enumerate(lines) if "FISH" in l)
section = lines[start_idx+1:end_idx]
print("\n".join(section))

import pdfplumber
import pandas as pd

def extract_section_between(pdf_path, start_word, end_word, page_num=1):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num - 1]
        chars = page.chars  # every character with x/y coords

    # Helper: find y positions of our target words built from letters
    def find_word_y(chars, target):
        target = target.upper()
        text = "".join(c["text"].upper() for c in chars)
        if not all(letter in text for letter in target):
            return None
        # find sequence of letters visually aligned
        letters = [c for c in chars if c["text"].upper() in target]
        # cluster by line (y coord)
        y_map = {}
        for c in letters:
            y_key = round(c["top"], -1)
            y_map.setdefault(y_key, []).append(c["text"])
        for y, seq in y_map.items():
            joined = "".join(seq)
            if target in joined:
                return y
        return None

    start_y = find_word_y(chars, start_word)
    end_y = find_word_y(chars, end_word)
    if start_y is None or end_y is None:
        raise ValueError("Couldn't locate start or end word — likely letter-split text.")

    if start_y > end_y:
        start_y, end_y = end_y, start_y

    section_chars = [c for c in chars if start_y < c["top"] < end_y]

    # group by y coord into lines
    lines = {}
    for c in section_chars:
        y_key = round(c["top"], -1)
        lines.setdefault(y_key, []).append(c)

    data = []
    for y, line_chars in sorted(lines.items()):
        sorted_line = sorted(line_chars, key=lambda x: x["x0"])
        data.append("".join(ch["text"] for ch in sorted_line))
    return data

section = extract_section_between("../data/raw/2025/2025-07-30.pdf", "RICE", "FISH", page_num=2)
print("\n".join(section[:30]))

Wholesale and Retail Prices: Selected Food Commodities - 30 July 2025
Wholesale Prices Retail Prices
Item Unit
Pettah Dambulla Pettah Dambulla Narahenpita
Yesterday Today Yesterday Today Yesterday Today Yesterday Today Yesterday Today
V E G E T A B L E S
Beans Rs./kg 6 50.00 6 00.00 4 50.00 4 75.00 7 00.00 6 50.00 4 80.00 5 05.00 7 00.00 7 00.00
Carrot Rs./kg 2 50.00 3 00.00 2 05.00 2 90.00 3 00.00 3 50.00 2 35.00 3 20.00 4 00.00 4 00.00
Cabbage Rs./kg 1 30.00 1 50.00 1 15.00 1 25.00 1 80.00 2 00.00 1 45.00 1 55.00 3 20.00 3 20.00
Tomato Rs./kg 1 20.00 1 20.00 9 0.00 9 0.00 1 70.00 1 70.00 1 20.00 1 20.00 2 00.00 2 00.00
Brinjal Rs./kg 3 00.00 3 00.00 2 25.00 2 40.00 3 50.00 3 50.00 2 55.00 2 70.00 4 80.00 4 80.00
Pumpkin Rs./kg 8 0.00 8 0.00 5 5.00 5 5.00 1 20.00 1 20.00 8 5.00 8 5.00 1 20.00 1 20.00
Snake gourd Rs./kg 1 30.00 1 30.00 9 0.00 9 0.00 1 80.00 1 80.00 1 20.00 1 20.00 3 60.00 3 60.00
Green Chilli Rs./kg 2 00.00 2 00.00 2 40.00 2 25.00 2 50.00 2 50.00 2 70.00 2 55.00 6 00.0

StopIteration: 

In [28]:
import pdfplumber
import pandas as pd

def extract_section_between(pdf_path, start_word, end_word, page_num=1):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num - 1]
        chars = page.chars  # each letter with x/y coords

    def find_line_y(chars, target_letters):
        """Find approximate y position of a target word built from its letters."""
        target_letters = list(target_letters.upper())
        y_map = {}
        for c in chars:
            y_key = round(c["top"], -1)
            y_map.setdefault(y_key, []).append(c["text"].upper())

        for y, letters in y_map.items():
            joined = "".join(letters)
            if all(t in joined for t in target_letters):
                return y
        return None

    start_y = find_line_y(chars, start_word)
    end_y = find_line_y(chars, end_word)

    if start_y is None or end_y is None:
        raise ValueError("Couldn’t locate start or end word — text likely letter-split.")

    # Ensure top < bottom
    if start_y > end_y:
        start_y, end_y = end_y, start_y

    # Filter characters between those Y ranges
    section_chars = [c for c in chars if start_y < c["top"] < end_y]

    # Group chars into lines by Y position
    lines = {}
    for c in section_chars:
        y_key = round(c["top"], -1)
        lines.setdefault(y_key, []).append(c)

    # Sort each line horizontally and build text
    data = []
    for y, line_chars in sorted(lines.items()):
        sorted_line = sorted(line_chars, key=lambda x: x["x0"])
        line_text = "".join(ch["text"] for ch in sorted_line)
        data.append(line_text.strip())
    return data

# ─────────────────────────────
# TEST IT HERE
# ─────────────────────────────
pdf_path = "../data/raw/2025/2025-07-30.pdf"  # change path as needed
section = extract_section_between(pdf_path, "RICE", "FISH", page_num=2)

print("\n".join(section[:30]))  # show first 30 lines


In [29]:
import pdfplumber

pdf_path = "../data/raw/2025/2025-07-30.pdf"
page_num = 2

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[page_num - 1]
    chars = page.chars

# Group characters by approximate line
y_map = {}
for c in chars:
    y_key = round(c["top"], -1)
    y_map.setdefault(y_key, []).append(c["text"])

for y, texts in sorted(y_map.items()):
    line = "".join(texts)
    if "R" in line or "I" in line or "C" in line or "E" in line:
        print(y, line)

30.0 Wholesale and Retail Prices: Selected Food Commodities   -30 July 2025
50.0 Wholesale PricesRetail Prices
60.0 PettahDambulla NarahenpitaItemUnitPettahDambulla 
90.0 V  E  G  E  T  A  B  L  E  S
110.0 BeansRs./kg650.00            600.00        450.00           475.00        700.00           650.00         480.00           505.00        700.00           700.00          
120.0 CarrotRs./kg250.00            300.00        205.00           290.00        300.00           350.00         235.00           320.00        400.00           400.00          
130.0 CabbageRs./kg130.00            150.00        115.00           125.00        180.00           200.00         145.00           155.00        320.00           320.00          
150.0 TomatoRs./kg120.00            120.00        90.00             90.00          170.00           170.00         120.00           120.00        200.00           200.00          
160.0 BrinjalRs./kg300.00            300.00        225.00           240.00        350.

In [37]:
import pdfplumber
import pandas as pd

def extract_section_between(pdf_path, start_letters, end_letters, page_num=2):
    with pdfplumber.open(pdf_path) as pdf:
        page = pdf.pages[page_num - 1]
        chars = page.chars

    # Group by approximate line y-position
    y_map = {}
    for c in chars:
        y_key = round(c["top"], -1)
        y_map.setdefault(y_key, []).append(c["text"])
    
    # Rebuild text lines
    lines = []
    for y, texts in sorted(y_map.items()):
        line_text = "".join(texts).replace("\xa0", "").strip()
        lines.append((y, line_text))
    
    # Find start and end positions
    def find_line_y(target):
        for y, text in lines:
            if all(ch in text for ch in target.replace(" ", "")):  # handles spaced "R I C E"
                return y
        return None

    start_y = find_line_y(start_letters)
    end_y = find_line_y(end_letters)
    if not start_y or not end_y:
        raise ValueError("Could not find start or end header.")

    # Collect lines in between
    section = [txt for y, txt in lines if start_y < y < end_y]
    return section

# ─── Run test ───────────────────────────────
pdf_path = "../data/raw/2025/2025-07-30.pdf"
rice_section = extract_section_between(pdf_path, "RICE", "FISH", page_num=2)

print("\n".join(rice_section))

Marandagahamula
SambaRs./kg235.00            235.00        234.00           234.00        240.00           240.00         244.00           244.00        240.00           240.00
NaduRs./kg213.00            213.00        208.00           208.00        230.00           230.00         235.00           235.00        230.00           230.00
Kekulu (White)Rs./kg203.00            203.00        200.00           198.00        220.00           220.00         220.00           220.00        220.00           220.00
Kekulu (Red)Rs./kg
203.00            203.00        195.00           195.00        220.00           220.00         220.00           220.00        210.00           210.00
Ponni Samba (Imp)Rs./kg230.00            230.00        n.a.n.a.233.00           233.00         n.a.n.a.
Nadu (Imp)Rs./kg210.00            210.00        n.a.n.a.220.00           220.00         n.a.n.a.
Kekulu (White) (Imp)Rs./kg
208.00            208.00        n.a.n.a.220.00           220.00         210.00           210.00


In [41]:
import re
import pandas as pd

def parse_price_section(section_lines):
    """Convert raw text lines (e.g. from RICE section) into a clean DataFrame."""
    merged_lines = []
    for line in section_lines:
        if not line.strip() or "Marandagahamula" in line:
            continue
        # Merge multi-line items (like Kekulu (Red))
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        elif merged_lines:
            merged_lines[-1] += " " + line

    parsed_rows = []
    for line in merged_lines:
        clean = line.replace(",", "").replace("…", "").replace("—", "-")

        # FIXED REGEX: keep n.a. together, also Rs./kg etc.
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|[0-9]+\.?[0-9]*",
            clean,
        )

        if len(tokens) < 3:
            continue

        # find where Rs./kg, Rs./Each, etc. appears
        try:
            idx_unit = next(i for i, t in enumerate(tokens) if "Rs" in t or "rs" in t)
        except StopIteration:
            continue

        item = " ".join(tokens[:idx_unit])
        unit = tokens[idx_unit]
        values = tokens[idx_unit + 1:]

        parsed_rows.append([item, unit] + values)

    # Pad to equal column count
    max_len = max(len(r) for r in parsed_rows)
    for r in parsed_rows:
        r += [None] * (max_len - len(r))

    cols = ["Item", "Unit"] + [f"Col{i}" for i in range(1, max_len - 1)]
    df = pd.DataFrame(parsed_rows, columns=cols)

    # Convert n.a. to NaN, numbers to floats
    df = df.replace({"n.a.": None, "N.A.": None})
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

In [42]:
pdf_path = "../data/raw/2025/2025-07-30.pdf"
rice_section = extract_section_between(pdf_path, "RICE", "FISH", page_num=2)

df_rice = parse_price_section(rice_section)
print(df_rice)
df_rice.to_csv("rice_clean.csv", index=False)
print("\n✅ Saved to rice_clean.csv")


             Item       Unit  Col1   Col2   Col3   Col4   Col5   Col6   Col7  \
0                    SambaRs   NaN  235.0  235.0  234.0  234.0  240.0  240.0   
1                     NaduRs   NaN  213.0  213.0  208.0  208.0  230.0  230.0   
2          Kekulu  (White)Rs   NaN  203.0  203.0  200.0  198.0  220.0  220.0   
3          Kekulu    (Red)Rs   NaN  203.0  203.0  195.0  195.0  220.0  220.0   
4     Ponni Samba    (Imp)Rs   NaN  230.0  230.0    NaN    NaN  233.0  233.0   
5            Nadu    (Imp)Rs   NaN  210.0  210.0    NaN    NaN  220.0  220.0   
6  Kekulu (White)    (Imp)Rs   NaN  208.0  208.0    NaN    NaN  220.0  220.0   

    Col8   Col9  Col10  Col11  
0  244.0  244.0  240.0  240.0  
1  235.0  235.0  230.0  230.0  
2  220.0  220.0  220.0  220.0  
3  220.0  220.0  210.0  210.0  
4    NaN    NaN    NaN    NaN  
5    NaN    NaN    NaN    NaN  
6  210.0  210.0    NaN    NaN  

✅ Saved to rice_clean.csv


In [43]:
df_rice.head(30)

,Item,Unit,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10,Col11
0,,SambaRs,NaN,235.0,235.0,234.0,234.0,240.0,240.0,244.0,244.0,240.0,240.0
1,,NaduRs,NaN,213.0,213.0,208.0,208.0,230.0,230.0,235.0,235.0,230.0,230.0
2,Kekulu,(White)Rs,NaN,203.0,203.0,200.0,198.0,220.0,220.0,220.0,220.0,220.0,220.0
3,Kekulu,(Red)Rs,NaN,203.0,203.0,195.0,195.0,220.0,220.0,220.0,220.0,210.0,210.0
4,Ponni Samba,(Imp)Rs,NaN,230.0,230.0,NaN,NaN,233.0,233.0,NaN,NaN,NaN,NaN
5,Nadu,(Imp)Rs,NaN,210.0,210.0,NaN,NaN,220.0,220.0,NaN,NaN,NaN,NaN
6,Kekulu (White),(Imp)Rs,NaN,208.0,208.0,NaN,NaN,220.0,220.0,210.0,210.0,NaN,NaN


In [57]:
import re
import pandas as pd

def parse_price_section(section_lines):
    """Convert text section (like RICE) into a clean structured DataFrame."""
    merged_lines = []
    for line in section_lines:
        if not line.strip() or "Marandagahamula" in line:
            continue
        # merge broken lines (e.g., Kekulu (Red))
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        elif merged_lines:
            merged_lines[-1] += " " + line

    parsed_rows = []

    for line in merged_lines:
        # ── Basic cleanup ─────────────────────────────
        clean = line.replace(",", "").replace("…", "").replace("—", "-")

        # 💡 Inject missing space before Rs. (e.g. "SambaRs./kg" → "Samba Rs./kg")
        clean = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean)

        # ✨ Fix glued prices like "130.00132.00" → "130.00 132.00"
        clean = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean)

        # ── Tokenize text ─────────────────────────────
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean,
        )

        if len(tokens) < 3:
            continue

        # find Rs./something as the unit
        try:
            idx_unit = next(i for i, t in enumerate(tokens) if "Rs" in t)
        except StopIteration:
            continue

        item = " ".join(tokens[:idx_unit])
        unit = tokens[idx_unit]
        values = tokens[idx_unit + 1:]

        # ── FIX: handle glued or partial decimals robustly ─────────────────
        split_values = []
        for v in values:
            # Extract all valid numbers inside each token
            nums = re.findall(r"\d+\.\d{1,2}", v)
            if nums:
                split_values.extend(nums)
            elif v.lower() == "n.a.":
                split_values.append(None)
            else:
                split_values.append(v)

        parsed_rows.append([item.strip(), unit.strip()] + split_values)

    # ── Pad rows for consistent DataFrame shape ─────────────────────────
    if not parsed_rows:
        return pd.DataFrame(columns=["Item", "Unit"])
    max_len = max(len(r) for r in parsed_rows)
    for r in parsed_rows:
        r += [None] * (max_len - len(r))

    df = pd.DataFrame(parsed_rows, columns=["Item", "Unit"] + [f"Col{i}" for i in range(1, max_len - 1)])

    # ── Clean leftover noise ─────────────────────────
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()

    # ── Catch hidden Rs inside Item (e.g., "Samba Rs./kg") ──────────────
    mask = (df["Unit"].eq("") | df["Unit"].isna()) & df["Item"].str.contains("Rs", na=False)
    for i, row in df.loc[mask].iterrows():
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", row["Item"])
        if match:
            df.at[i, "Item"] = match.group(1).strip()
            df.at[i, "Unit"] = match.group(2).strip()

    # ── Replace n.a. with NaN and convert to numbers ─────────────────────
    df = df.replace({"n.a.": None, "N.A.": None})
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df

In [59]:
pdf_path = "../data/raw/2022/2022-01-04.pdf"
rice_section = extract_section_between(pdf_path, "RICE", "FISH", page_num=2)

df_rice = parse_price_section(rice_section)
print(df_rice)
df_rice.to_csv("rice_clean.csv", index=False)
print("\n✅ Saved to rice_clean.csv")
display(df_rice.head(30))

                   Item    Unit   Col1   Col2   Col3   Col4   Col5   Col6  \
0                 Samba  Rs./kg  150.0  152.0  155.0  155.0    NaN    NaN   
1                  Nadu  Rs./kg  138.0  141.0  140.0  140.0    NaN    NaN   
2        Kekulu (White)  Rs./kg  139.0  137.0  128.0  128.0    NaN    NaN   
3          Kekulu (Red)  Rs./kg  128.0  133.0  123.0  138.0    NaN    NaN   
4     Ponni Samba (Imp)  Rs./kg  127.0  127.0  121.0  121.0  130.0  130.0   
5            Nadu (Imp)  Rs./kg  125.0  120.0  116.0  116.0    NaN    NaN   
6  Kekulu (White) (Imp)  Rs./kg  118.0  122.0  104.0  104.0  120.0    NaN   

    Col7   Col8   Col9  Col10  
0  155.0  175.0  170.0  165.0  
1  145.0  160.0  145.0  160.0  
2  133.0  145.0  165.0  150.0  
3  120.0  145.0  140.0  130.0  
4  140.0  140.0    NaN    NaN  
5  120.0  130.0    NaN    NaN  
6  115.0  130.0    NaN    NaN  

✅ Saved to rice_clean.csv


,Item,Unit,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,Samba,Rs./kg,150.0,152.0,155.0,155.0,NaN,NaN,155.0,175.0,170.0,165.0
1,Nadu,Rs./kg,138.0,141.0,140.0,140.0,NaN,NaN,145.0,160.0,145.0,160.0
2,Kekulu (White),Rs./kg,139.0,137.0,128.0,128.0,NaN,NaN,133.0,145.0,165.0,150.0
3,Kekulu (Red),Rs./kg,128.0,133.0,123.0,138.0,NaN,NaN,120.0,145.0,140.0,130.0
4,Ponni Samba (Imp),Rs./kg,127.0,127.0,121.0,121.0,130.0,130.0,140.0,140.0,NaN,NaN
5,Nadu (Imp),Rs./kg,125.0,120.0,116.0,116.0,NaN,NaN,120.0,130.0,NaN,NaN
6,Kekulu (White) (Imp),Rs./kg,118.0,122.0,104.0,104.0,120.0,NaN,115.0,130.0,NaN,NaN


In [60]:
def get_dynamic_column_names(section_lines):
    text = " ".join(section_lines[:40]).lower()
    columns = ["item", "unit"]

    # default locations
    wholesale_locs = ["pettah", "dambulla"]
    retail_locs = ["pettah", "dambulla", "narahenpita"]

    # detect Marandagahamula case
    if "marandagahamula" in text:
        wholesale_locs.insert(0, "marandagahamula")

    for loc in wholesale_locs:
        columns += [f"wholesale_{loc}_yesterday", f"wholesale_{loc}_today"]

    for loc in retail_locs:
        columns += [f"retail_{loc}_yesterday", f"retail_{loc}_today"]

    return columns


In [ ]:
cols = get_dynamic_column_names(section_lines)
cols = cols[:len(df_rice.columns)]  # trim if fewer columns exist
df_rice.columns = cols

In [66]:
import re
import pandas as pd

# ──────────────────────────────────────────────
# 1️⃣ Parse RICE section into structured DataFrame
# ──────────────────────────────────────────────
def parse_price_section(section_lines):
    merged_lines = []
    for line in section_lines:
        if not line.strip() or "Marandagahamula" in line:
            continue
        # merge broken item lines
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        elif merged_lines:
            merged_lines[-1] += " " + line

    parsed_rows = []

    for line in merged_lines:
        # clean up text
        clean = line.replace(",", "").replace("…", "").replace("—", "-")
        clean = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean)
        # fix glued numbers like "130.00132.00"
        clean = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean)

        # tokenize text into words and numbers
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean,
        )
        if len(tokens) < 3:
            continue

        # detect Rs./kg as unit marker
        try:
            idx_unit = next(i for i, t in enumerate(tokens) if "Rs" in t)
        except StopIteration:
            continue

        item = " ".join(tokens[:idx_unit])
        unit = tokens[idx_unit]
        values = tokens[idx_unit + 1:]

        split_values = []
        for v in values:
            nums = re.findall(r"\d+\.\d{1,2}", v)
            if nums:
                split_values.extend(nums)
            elif v.lower() == "n.a.":
                split_values.append(None)
            else:
                split_values.append(v)

        parsed_rows.append([item.strip(), unit.strip()] + split_values)

    if not parsed_rows:
        return pd.DataFrame(columns=["Item", "Unit"])

    # normalize row lengths
    max_len = max(len(r) for r in parsed_rows)
    for r in parsed_rows:
        r += [None] * (max_len - len(r))

    df = pd.DataFrame(
        parsed_rows,
        columns=["Item", "Unit"] + [f"Col{i}" for i in range(1, max_len - 1)],
    )

    # cleanup columns
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()

    # fix cases like "Samba Rs./kg"
    mask = (df["Unit"].eq("") | df["Unit"].isna()) & df["Item"].str.contains("Rs", na=False)
    for i, row in df.loc[mask].iterrows():
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", row["Item"])
        if match:
            df.at[i, "Item"] = match.group(1).strip()
            df.at[i, "Unit"] = match.group(2).strip()

    # convert to numeric
    df = df.replace({"n.a.": None, "N.A.": None})
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


# ──────────────────────────────────────────────
# 2️⃣ Dynamic column name generator
# ──────────────────────────────────────────────
def get_dynamic_column_names(section_lines):
    text = " ".join(section_lines[:40]).lower()
    columns = ["item", "unit"]

    # Default locations
    wholesale_locs = ["pettah"]
    retail_locs = ["pettah", "dambulla", "narahenpita"]

    # If Marandagahamula is mentioned, insert it right after Pettah
    if "marandagahamula" in text:
        wholesale_locs += ["marandagahamula"]

    # # Add Dambulla *after* Marandagahamula (or Pettah if not present)
    # wholesale_locs += ["dambulla"]

    # Build wholesale + retail headers
    for loc in wholesale_locs:
        columns += [f"wholesale_{loc}_yesterday", f"wholesale_{loc}_today"]

    for loc in retail_locs:
        columns += [f"retail_{loc}_yesterday", f"retail_{loc}_today"]

    return columns



# ──────────────────────────────────────────────
# 3️⃣ Combine both steps for one clean call
# ──────────────────────────────────────────────
def extract_and_parse_rice(pdf_path, start="RICE", end="FISH", page_num=2):
    """Extracts RICE section and returns a labeled DataFrame."""
    section_lines = extract_section_between(pdf_path, start, end, page_num)
    df = parse_price_section(section_lines)

    cols = get_dynamic_column_names(section_lines)
    cols = cols[:len(df.columns)]
    df.columns = cols

    return df


# ──────────────────────────────────────────────
# 4️⃣ Example run
# ──────────────────────────────────────────────
pdf_path = "../data/raw/2023/2023-01-04.pdf"
df_rice = extract_and_parse_rice(pdf_path)

df_rice.to_csv("rice_clean.csv", index=False)
print("\n✅ Saved to rice_clean.csv")
display(df_rice.head(15))


✅ Saved to rice_clean.csv


,item,unit,wholesale_pettah_yesterday,wholesale_pettah_today,wholesale_marandagahamula_yesterday,wholesale_marandagahamula_today,retail_pettah_yesterday,retail_pettah_today,retail_dambulla_yesterday,retail_dambulla_today,retail_narahenpita_yesterday,retail_narahenpita_today
0,Samba,Rs./kg,213.0,213.0,198.0,198.0,220.0,220.0,240.0,240.0,225.0,225.0
1,Nadu,Rs./kg,203.0,203.0,193.0,193.0,215.0,215.0,208.0,208.0,220.0,220.0
2,Kekulu (White),Rs./kg,193.0,195.0,183.0,183.0,210.0,210.0,190.0,190.0,210.0,210.0
3,Kekulu (Red),Rs./kg,200.0,200.0,205.0,205.0,215.0,215.0,NaN,NaN,210.0,210.0
4,Ponni Samba (Imp),Rs./kg,206.0,203.0,194.0,195.0,210.0,210.0,230.0,230.0,NaN,NaN
5,Nadu (Imp),Rs./kg,178.0,178.0,174.0,174.0,200.0,200.0,205.0,205.0,NaN,NaN
6,Kekulu (White) (Imp),Rs./kg,173.0,170.0,170.0,170.0,200.0,200.0,195.0,195.0,NaN,NaN


# Extraction

In [67]:
pdf_path = "../data/raw/2023/2023-01-04.pdf"
rice_section = extract_section_between(pdf_path, "RICE", "FISH", page_num=2)

print("\n--- RAW EXTRACTED LINES ---")
for line in rice_section[:10]:
    print(line)


--- RAW EXTRACTED LINES ---
Marandagahamula
SambaRs./kg213.00          213.00          198.00          198.00          220.00          220.00          240.00          240.00          225.00          225.00
NaduRs./kg203.00          203.00          193.00          193.00          215.00          215.00          208.00          208.00          220.00          220.00
Kekulu (White)Rs./kg193.00          195.00          183.00          183.00          210.00          210.00          190.00          190.00          210.00          210.00
Kekulu (Red)Rs./kg200.00          200.00          205.00          205.00          215.00          215.00          n.a.n.a.210.00          210.00
Ponni Samba (Imp)Rs./kg206.00          203.00          194.00          195.00          210.00          210.00          230.00          230.00
Nadu (Imp)Rs./kg
178.00          178.00          174.00          174.00          200.00          200.00          205.00          205.00
Kekulu (White) (Imp)Rs./kg173.00      

In [68]:
df_rice = parse_price_section(rice_section)
display(df_rice.head(10))

,Item,Unit,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,Samba,Rs./kg,213.0,213.0,198.0,198.0,220.0,220.0,240.0,240.0,225.0,225.0
1,Nadu,Rs./kg,203.0,203.0,193.0,193.0,215.0,215.0,208.0,208.0,220.0,220.0
2,Kekulu (White),Rs./kg,193.0,195.0,183.0,183.0,210.0,210.0,190.0,190.0,210.0,210.0
3,Kekulu (Red),Rs./kg,200.0,200.0,205.0,205.0,215.0,215.0,NaN,NaN,210.0,210.0
4,Ponni Samba (Imp),Rs./kg,206.0,203.0,194.0,195.0,210.0,210.0,230.0,230.0,NaN,NaN
5,Nadu (Imp),Rs./kg,178.0,178.0,174.0,174.0,200.0,200.0,205.0,205.0,NaN,NaN
6,Kekulu (White) (Imp),Rs./kg,173.0,170.0,170.0,170.0,200.0,200.0,195.0,195.0,NaN,NaN


In [69]:
import re
import pandas as pd

# ──────────────────────────────────────────────
# 1️⃣ Parse RICE section into structured DataFrame
# ──────────────────────────────────────────────
def parse_price_section(section_lines):
    merged_lines = []
    for line in section_lines:
        if not line.strip() or "Marandagahamula" in line:
            continue
        # merge broken item lines
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        elif merged_lines:
            merged_lines[-1] += " " + line

    parsed_rows = []

    for line in merged_lines:
        # clean up text
        clean = line.replace(",", "").replace("…", "").replace("—", "-")
        clean = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean)
        # fix glued numbers like "130.00132.00"
        clean = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean)

        # tokenize text into words and numbers
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean,
        )
        if len(tokens) < 3:
            continue

        # detect Rs./kg as unit marker
        try:
            idx_unit = next(i for i, t in enumerate(tokens) if "Rs" in t)
        except StopIteration:
            continue

        item = " ".join(tokens[:idx_unit])
        unit = tokens[idx_unit]
        values = tokens[idx_unit + 1:]

        split_values = []
        for v in values:
            nums = re.findall(r"\d+\.\d{1,2}", v)
            if nums:
                split_values.extend(nums)
            elif v.lower() == "n.a.":
                split_values.append(None)
            else:
                split_values.append(v)

        parsed_rows.append([item.strip(), unit.strip()] + split_values)

    if not parsed_rows:
        return pd.DataFrame(columns=["Item", "Unit"])

    # normalize row lengths
    max_len = max(len(r) for r in parsed_rows)
    for r in parsed_rows:
        r += [None] * (max_len - len(r))

    df = pd.DataFrame(
        parsed_rows,
        columns=["Item", "Unit"] + [f"Col{i}" for i in range(1, max_len - 1)],
    )

    # cleanup columns
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()

    # fix cases like "Samba Rs./kg"
    mask = (df["Unit"].eq("") | df["Unit"].isna()) & df["Item"].str.contains("Rs", na=False)
    for i, row in df.loc[mask].iterrows():
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", row["Item"])
        if match:
            df.at[i, "Item"] = match.group(1).strip()
            df.at[i, "Unit"] = match.group(2).strip()

    # convert to numeric
    df = df.replace({"n.a.": None, "N.A.": None})
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    return df


# ──────────────────────────────────────────────
# 2️⃣ Dynamic column name generator
# ──────────────────────────────────────────────
def get_dynamic_column_names(section_lines):
    text = " ".join(section_lines[:40]).lower()
    columns = ["item", "unit"]

    # Default locations
    wholesale_locs = ["pettah"]
    retail_locs = ["pettah", "dambulla", "narahenpita"]

    # If Marandagahamula is mentioned, insert it right after Pettah
    if "marandagahamula" in text:
        wholesale_locs += ["marandagahamula"]

    # # Add Dambulla *after* Marandagahamula (or Pettah if not present)
    # wholesale_locs += ["dambulla"]

    # Build wholesale + retail headers
    for loc in wholesale_locs:
        columns += [f"wholesale_{loc}_yesterday", f"wholesale_{loc}_today"]

    for loc in retail_locs:
        columns += [f"retail_{loc}_yesterday", f"retail_{loc}_today"]

    return columns



# ──────────────────────────────────────────────
# 3️⃣ Combine both steps for one clean call
# ──────────────────────────────────────────────
def extract_and_parse_rice(pdf_path, start="RICE", end="FISH", page_num=2):
    """Extracts RICE section and returns a labeled DataFrame."""
    section_lines = extract_section_between(pdf_path, start, end, page_num)
    df = parse_price_section(section_lines)

    cols = get_dynamic_column_names(section_lines)
    cols = cols[:len(df.columns)]
    df.columns = cols

    return df


# ──────────────────────────────────────────────
# 4️⃣ Example run
# ──────────────────────────────────────────────
pdf_path = "../data/raw/2023/2023-01-04.pdf"
df_rice = extract_and_parse_rice(pdf_path)

df_rice.to_csv("rice_clean.csv", index=False)
print("\n✅ Saved to rice_clean.csv")
display(df_rice.head(15))

SyntaxError: incomplete input (4290173342.py, line 4)

In [85]:
import re
import pandas as pd

# ──────────────────────────────────────────────
# Helper: pad missing middle columns intelligently
# ──────────────────────────────────────────────
def normalize_price_row(values, expected=10, pad_index=6):
    """
    Ensures consistent number of numeric slots per row.
    Pads missing Nones *after the wholesale group* (default index = 6).
    This aligns missing retail columns correctly.
    """
    values = list(values)
    if len(values) < expected:
        diff = expected - len(values)
        insert_at = min(pad_index, len(values))
        values[insert_at:insert_at] = [None] * diff
    return values


# ──────────────────────────────────────────────
# Main parser
# ──────────────────────────────────────────────
def parse_price_section(section_lines):
    """
    Convert a price section (e.g., 'RICE') into a clean structured DataFrame.
    Handles broken lines, glued numbers, and invisible column gaps.
    """

    # Step 1: merge broken lines
    merged_lines = []
    for line in section_lines:
        if not line.strip() or "Marandagahamula" in line:
            continue

        # if starts with a letter, it's a new row
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        elif merged_lines:
            merged_lines[-1] += " " + line  # continuation of previous item

    parsed_rows = []

    # Step 2: clean and tokenize each line
    for line in merged_lines:
        clean = (
            line.replace(",", "")
                .replace("...", "")
                .replace("—", "-")
        )
        clean = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean)
        clean = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean)

        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean
        )
        if len(tokens) < 3:
            continue

        # find Rs./kg or Rs./Ltr as the unit marker
        try:
            idx_unit = next(i for i, t in enumerate(tokens) if "Rs" in t)
        except StopIteration:
            continue

        item = " ".join(tokens[:idx_unit]).strip()
        unit = tokens[idx_unit].strip()
        values = tokens[idx_unit + 1:]

        # Step 3: extract numbers and mark blanks
        split_values = []
        for v in values:
            nums = re.findall(r"\d+\.\d{1,2}", v)
            if nums:
                split_values.extend(nums)
            elif v.lower() == "n.a.":
                split_values.append(None)
            else:
                split_values.append(None)

        # Step 4: pad missing mid columns (expected 10 cols, pad after col 6)
        split_values = normalize_price_row(split_values, expected=10, pad_index=6)

        parsed_rows.append([item, unit] + split_values)

    # Step 5: build DataFrame
    if not parsed_rows:
        return pd.DataFrame(columns=["Item", "Unit"])

    max_len = max(len(r) for r in parsed_rows)
    for r in parsed_rows:
        r += [None] * (max_len - len(r))

    df = pd.DataFrame(
        parsed_rows,
        columns=["Item", "Unit"] + [f"Col{i}" for i in range(1, max_len - 1)]
    )

    # Step 6: cleanup units and items
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()

    # fix cases where Rs is stuck in Item
    mask = (df["Unit"].eq("") | df["Unit"].isna()) & df["Item"].str.contains("Rs", na=False)
    for i, row in df.loc[mask].iterrows():
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", row["Item"])
        if match:
            df.at[i, "Item"] = match.group(1).strip()
            df.at[i, "Unit"] = match.group(2).strip()

    # Step 7: convert prices to numeric
    df = df.replace({"n.a.": None, "N.A.": None})
    for col in df.columns[2:]:
        df[col] = pd.to_numeric(df[col], errors="coerce")

    # ✅ Step 8: Return DataFrame
    return df


# ──────────────────────────────────────────────
# Extract + parse function
# ──────────────────────────────────────────────
def extract_and_parse_rice(pdf_path, start="RICE", end="FISH", page_num=2):
    """Extracts RICE section and parses into a DataFrame."""
    section_lines = extract_section_between(pdf_path, start, end, page_num)
    df = parse_price_section(section_lines)
    return df


# ──────────────────────────────────────────────
# Example run
# ──────────────────────────────────────────────
pdf_path = "../data/raw/2025/2025-01-03.pdf"
df_rice = extract_and_parse_rice(pdf_path)

print("\n✅ DataFrame created successfully!")
display(df_rice.head(10))






✅ DataFrame created successfully!


,Item,Unit,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,Samba,Rs./kg,235.0,235.0,239.0,239.0,NaN,NaN,240.0,240.0,240.0,240.0
1,Nadu,Rs./kg,220.0,220.0,229.0,229.0,NaN,NaN,225.0,NaN,230.0,230.0
2,Kekulu (White),Rs./kg,215.0,215.0,217.0,217.0,NaN,NaN,225.0,NaN,NaN,NaN
3,Kekulu (Red),Rs./kg,215.0,215.0,NaN,NaN,NaN,NaN,NaN,NaN,220.0,220.0
4,Ponni Samba (Imp),Rs./kg,228.0,228.0,NaN,NaN,240.0,240.0,NaN,NaN,240.0,240.0
5,Nadu (Imp),Rs./kg,220.0,220.0,NaN,NaN,230.0,230.0,NaN,NaN,NaN,NaN
6,Kekulu (White) (Imp),Rs./kg,215.0,213.0,NaN,NaN,230.0,230.0,NaN,NaN,220.0,220.0


In [88]:
import re
import pandas as pd

# ═══════════════════════════════════════════════════════════════════
# PART 1: Helper Function - Fix Missing Columns
# ═══════════════════════════════════════════════════════════════════

def fix_missing_columns(price_list, total_columns=10, insert_position=6):
    """
    Makes sure each row has the same number of price columns.
    
    Why? Sometimes data has missing columns in the middle (like missing retail prices).
    We need to add empty spaces (None) in the right place.
    
    Example:
        Input:  [130.00, 132.00, 135.00]  (only 3 prices, but we need 10)
        Output: [130.00, 132.00, 135.00, None, None, None, None, None, None, None]
        
        The None values are added after position 6 (after wholesale prices).
    
    Parameters:
        price_list: List of prices (may be incomplete)
        total_columns: How many columns we want in total (default: 10)
        insert_position: Where to add missing values (default: 6)
    
    Returns:
        A list with exactly 'total_columns' items
    """
    # Make a copy so we don't change the original
    price_list = list(price_list)
    
    # Check if we're missing columns
    if len(price_list) < total_columns:
        # Calculate how many columns are missing
        missing_count = total_columns - len(price_list)
        
        # Don't insert beyond what we have
        safe_position = min(insert_position, len(price_list))
        
        # Insert None values at the right position
        # This is like cutting the list and inserting empty spaces
        for i in range(missing_count):
            price_list.insert(safe_position, None)
    
    return price_list


# ═══════════════════════════════════════════════════════════════════
# PART 2: Main Parser - Convert Messy Text to Clean Table
# ═══════════════════════════════════════════════════════════════════

def parse_price_section(section_lines):
    """
    Takes messy price text and creates a neat table (DataFrame).
    
    What it does:
        1. Combines broken lines
        2. Cleans up the text
        3. Splits each line into: Item name, Unit, and Prices
        4. Creates a table with all the data
    
    Input example:
        ["Samba Rs./kg 130.00 132.00 135.00",
         "Nadu (White)",
         "Rs./kg 125.00 128.00"]
    
    Output: A pandas table with columns [Item, Unit, Col1, Col2, ...]
    """
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 1: Combine Lines That Belong Together
    # ─────────────────────────────────────────────────────────────────
    # Sometimes one item is split across multiple lines. We fix that here.
    
    merged_lines = []
    
    for line in section_lines:
        # Skip empty lines or lines with "Marandagahamula"
        if not line.strip() or "Marandagahamula" in line:
            continue
        
        # Does this line start with a letter? Then it's a NEW item
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        
        # Otherwise, add it to the PREVIOUS item
        elif len(merged_lines) > 0:
            merged_lines[-1] = merged_lines[-1] + " " + line
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 2: Process Each Line
    # ─────────────────────────────────────────────────────────────────
    
    parsed_rows = []  # Will store all our cleaned data
    
    for line in merged_lines:
        
        # ─── Clean up the text ───
        clean_line = line
        clean_line = clean_line.replace(",", "")      # Remove commas
        clean_line = clean_line.replace("...", "")    # Remove dots
        clean_line = clean_line.replace("—", "-")     # Fix dashes
        
        # Add space between letters and "Rs."
        # "SambaRs." becomes "Samba Rs."
        clean_line = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean_line)
        
        # Separate numbers that are stuck together
        # "130.00132.00" becomes "130.00 132.00"
        clean_line = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean_line)
        
        # ─── Break line into pieces (tokens) ───
        # Find: "n.a.", "Rs./kg", words, and decimal numbers
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean_line
        )
        
        # Need at least 3 pieces (item name, unit, one price)
        if len(tokens) < 3:
            continue  # Skip this line, not enough data
        
        # ─── Find the unit marker (Rs./kg or Rs./Ltr) ───
        unit_index = None
        for i, token in enumerate(tokens):
            if "Rs" in token:
                unit_index = i
                break
        
        # If no unit found, skip this line
        if unit_index is None:
            continue
        
        # ─── Split tokens into parts ───
        item_name = " ".join(tokens[:unit_index]).strip()  # Everything before "Rs./kg"
        unit = tokens[unit_index].strip()                  # "Rs./kg"
        value_tokens = tokens[unit_index + 1:]             # All prices after unit
        
        # ─────────────────────────────────────────────────────────────────
        # STEP 3: Extract Price Numbers
        # ─────────────────────────────────────────────────────────────────
        
        prices = []
        for value in value_tokens:
            # Find decimal numbers in this token
            numbers = re.findall(r"\d+\.\d{1,2}", value)
            
            if numbers:
                # Add all numbers found
                prices.extend(numbers)
            elif value.lower() == "n.a.":
                # "n.a." means missing data
                prices.append(None)
            else:
                # Unknown value, treat as missing
                prices.append(None)
        
        # ─────────────────────────────────────────────────────────────────
        # STEP 4: Fix Missing Columns
        # ─────────────────────────────────────────────────────────────────
        # Make sure we have exactly 10 price columns
        # Missing columns are added after position 6
        
        prices = fix_missing_columns(prices, total_columns=10, insert_position=6)
        
        # ─── Create row: [Item, Unit, Price1, Price2, ...] ───
        row = [item_name, unit] + prices
        parsed_rows.append(row)
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 5: Create the DataFrame (Table)
    # ─────────────────────────────────────────────────────────────────
    
    # If no data was found, return an empty table
    if len(parsed_rows) == 0:
        return pd.DataFrame(columns=["Item", "Unit"])
    
    # Find the longest row
    max_length = max(len(row) for row in parsed_rows)
    
    # Make all rows the same length by adding None to the end
    for row in parsed_rows:
        while len(row) < max_length:
            row.append(None)
    
    # Create column names: Item, Unit, Col1, Col2, Col3...
    num_price_cols = max_length - 2
    column_names = ["Item", "Unit"] + [f"Col{i}" for i in range(1, num_price_cols + 1)]
    
    # Create the DataFrame
    df = pd.DataFrame(parsed_rows, columns=column_names)
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 6: Clean Up Item Names and Units
    # ─────────────────────────────────────────────────────────────────
    
    # Remove "Rs" from end of item names
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    
    # Clean unit column (keep only letters, slash, dot)
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()
    
    # Fix cases where "Rs./kg" ended up in the Item column
    # Example: "Samba Rs./kg" should be split into "Samba" | "Rs./kg"
    
    # Find rows where Unit is empty but Item contains "Rs"
    has_problem = (df["Unit"] == "") | (df["Unit"].isna())
    has_problem = has_problem & df["Item"].str.contains("Rs", na=False)
    
    for index in df[has_problem].index:
        item_text = df.at[index, "Item"]
        
        # Try to split "Samba Rs./kg" into two parts
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", item_text)
        if match:
            df.at[index, "Item"] = match.group(1).strip()  # "Samba"
            df.at[index, "Unit"] = match.group(2).strip()  # "Rs./kg"
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 7: Convert Prices to Numbers
    # ─────────────────────────────────────────────────────────────────
    
    # Replace text "n.a." with actual None (missing value)
    df = df.replace({"n.a.": None, "N.A.": None})
    
    # Convert all price columns from text to numbers
    for col in df.columns[2:]:  # Skip Item and Unit columns
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    # ✅ Done! Return the clean table
    return df


# ═══════════════════════════════════════════════════════════════════
# PART 3: Extract Section from PDF and Parse It
# ═══════════════════════════════════════════════════════════════════

def extract_and_parse_rice(pdf_path, start_word="RICE", end_word="FISH", page_number=2):
    """
    Extracts the RICE section from a PDF and converts it to a table.
    
    This function does two things:
        1. Finds and extracts text between "RICE" and "FISH" in the PDF
        2. Parses that text into a clean DataFrame
    
    Parameters:
        pdf_path: Path to the PDF file
        start_word: Word that marks the beginning of the section (default: "RICE")
        end_word: Word that marks the end of the section (default: "FISH")
        page_number: Which page to look at (default: 2)
    
    Returns:
        A pandas DataFrame with the parsed price data
    """
    
    # NOTE: You need to define extract_section_between() function
    # That function should read the PDF and return lines of text
    section_lines = extract_section_between(pdf_path, start_word, end_word, page_number)
    
    # Parse the extracted lines into a DataFrame
    df = parse_price_section(section_lines)
    
    return df


# ═══════════════════════════════════════════════════════════════════
# PART 4: Example Usage (How to Run This Code)
# ═══════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # # Example 1: Test with sample data (no PDF needed)
    # print("=" * 60)
    # print("EXAMPLE 1: Testing with sample data")
    # print("=" * 60)
    
    # sample_lines = [
    #     "Samba Rs./kg 130.00 132.00 135.00 140.00",
    #     "Nadu (White)",
    #     "Rs./kg 125.00 128.00 130.00",
    #     "Red Nadu Rs./kg 120.00 122.00 n.a. 125.00",
    #     "Keeri Samba Rs./kg 140.00 145.00 150.00 155.00 160.00"
    # ]
    
    # result = parse_price_section(sample_lines)
    # print("\n📊 Parsed Table:")
    # print(result)
    # print("\n📋 Column Types:")
    # print(result.dtypes)
    
    # Example 2: Parse from actual PDF (uncomment to use)
    print("\n" + "=" * 60)
    print("EXAMPLE 2: Parsing from PDF")
    print("=" * 60)
    
    pdf_path = "../data/raw/2025/2025-01-03.pdf"
    df_rice = extract_and_parse_rice(pdf_path)
    
    print("\n✅ DataFrame created successfully!")
    display(df_rice.head(10))


EXAMPLE 2: Parsing from PDF

✅ DataFrame created successfully!


,Item,Unit,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,Samba,Rs./kg,235.0,235.0,239.0,239.0,NaN,NaN,240.0,240.0,240.0,240.0
1,Nadu,Rs./kg,220.0,220.0,229.0,229.0,NaN,NaN,225.0,NaN,230.0,230.0
2,Kekulu (White),Rs./kg,215.0,215.0,217.0,217.0,NaN,NaN,225.0,NaN,NaN,NaN
3,Kekulu (Red),Rs./kg,215.0,215.0,NaN,NaN,NaN,NaN,NaN,NaN,220.0,220.0
4,Ponni Samba (Imp),Rs./kg,228.0,228.0,NaN,NaN,240.0,240.0,NaN,NaN,240.0,240.0
5,Nadu (Imp),Rs./kg,220.0,220.0,NaN,NaN,230.0,230.0,NaN,NaN,NaN,NaN
6,Kekulu (White) (Imp),Rs./kg,215.0,213.0,NaN,NaN,230.0,230.0,NaN,NaN,220.0,220.0


In [92]:
import re
import pandas as pd

# ═══════════════════════════════════════════════════════════════════
# PART 1: Helper Function - Fix Missing Columns
# ═══════════════════════════════════════════════════════════════════

def fix_missing_columns(price_list, total_columns=10, insert_position=6):
    """
    Makes sure each row has the same number of price columns.
    
    Why? Sometimes data has missing columns in the middle (like missing retail prices).
    We need to add empty spaces (None) in the right place.
    
    Example:
        Input:  [130.00, 132.00, 135.00]  (only 3 prices, but we need 10)
        Output: [130.00, 132.00, 135.00, None, None, None, None, None, None, None]
        
        The None values are added after position 6 (after wholesale prices).
    
    Parameters:
        price_list: List of prices (may be incomplete)
        total_columns: How many columns we want in total (default: 10)
        insert_position: Where to add missing values (default: 6)
    
    Returns:
        A list with exactly 'total_columns' items
    """
    # Make a copy so we don't change the original
    price_list = list(price_list)
    
    # Check if we're missing columns
    if len(price_list) < total_columns:
        # Calculate how many columns are missing
        missing_count = total_columns - len(price_list)
        
        # Don't insert beyond what we have
        safe_position = min(insert_position, len(price_list))
        
        # Insert None values at the right position
        # This is like cutting the list and inserting empty spaces
        for i in range(missing_count):
            price_list.insert(safe_position, None)
    
    return price_list


# ═══════════════════════════════════════════════════════════════════
# PART 2: Main Parser - Convert Messy Text to Clean Table
# ═══════════════════════════════════════════════════════════════════

def parse_price_section(section_lines):
    """
    Takes messy price text and creates a neat table (DataFrame).
    
    What it does:
        1. Combines broken lines
        2. Cleans up the text
        3. Splits each line into: Item name, Unit, and Prices
        4. Creates a table with all the data
    
    Input example:
        ["Samba Rs./kg 130.00 132.00 135.00",
         "Nadu (White)",
         "Rs./kg 125.00 128.00"]
    
    Output: A pandas table with columns [Item, Unit, Col1, Col2, ...]
    """
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 1: Combine Lines That Belong Together
    # ─────────────────────────────────────────────────────────────────
    # Sometimes one item is split across multiple lines. We fix that here.
    
    merged_lines = []
    
    for line in section_lines:
        # Skip empty lines or lines with "Marandagahamula"
        if not line.strip() or "Marandagahamula" in line:
            continue
        
        # Does this line start with a letter? Then it's a NEW item
        if re.match(r"^[A-Za-z]", line):
            merged_lines.append(line)
        
        # Otherwise, add it to the PREVIOUS item
        elif len(merged_lines) > 0:
            merged_lines[-1] = merged_lines[-1] + " " + line
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 2: Process Each Line
    # ─────────────────────────────────────────────────────────────────
    
    parsed_rows = []  # Will store all our cleaned data
    
    for line in merged_lines:
        
        # ─── Clean up the text ───
        clean_line = line
        clean_line = clean_line.replace(",", "")      # Remove commas
        clean_line = clean_line.replace("...", "")    # Remove dots
        clean_line = clean_line.replace("—", "-")     # Fix dashes
        
        # Add space between letters and "Rs."
        # "SambaRs." becomes "Samba Rs."
        clean_line = re.sub(r"([a-zA-Z)])(Rs\.)", r"\1 \2", clean_line)
        
        # Separate numbers that are stuck together
        # "130.00132.00" becomes "130.00 132.00"
        clean_line = re.sub(r"(\d+\.\d{1,2})(?=\d+\.\d{1,2})", r"\1 ", clean_line)
        
        # ─── Break line into pieces (tokens) ───
        # Find: "n.a.", "Rs./kg", words, and decimal numbers
        tokens = re.findall(
            r"n\.a\.|N\.A\.|Rs\.\/[A-Za-z]+|[A-Za-z()]+|\d+\.\d{1,2}|\d+",
            clean_line
        )
        
        # Need at least 3 pieces (item name, unit, one price)
        if len(tokens) < 3:
            continue  # Skip this line, not enough data
        
        # ─── Find the unit marker (Rs./kg or Rs./Ltr) ───
        unit_index = None
        for i, token in enumerate(tokens):
            if "Rs" in token:
                unit_index = i
                break
        
        # If no unit found, skip this line
        if unit_index is None:
            continue
        
        # ─── Split tokens into parts ───
        item_name = " ".join(tokens[:unit_index]).strip()  # Everything before "Rs./kg"
        unit = tokens[unit_index].strip()                  # "Rs./kg"
        value_tokens = tokens[unit_index + 1:]             # All prices after unit
        
        # ─────────────────────────────────────────────────────────────────
        # STEP 3: Extract Price Numbers
        # ─────────────────────────────────────────────────────────────────
        
        prices = []
        for value in value_tokens:
            # Find decimal numbers in this token
            numbers = re.findall(r"\d+\.\d{1,2}", value)
            
            if numbers:
                # Add all numbers found
                prices.extend(numbers)
            elif value.lower() == "n.a.":
                # "n.a." means missing data
                prices.append(None)
            else:
                # Unknown value, treat as missing
                prices.append(None)
        
        # ─────────────────────────────────────────────────────────────────
        # STEP 4: Fix Missing Columns
        # ─────────────────────────────────────────────────────────────────
        # Make sure we have exactly 10 price columns
        # Missing columns are added after position 6
        
        prices = fix_missing_columns(prices, total_columns=10, insert_position=6)
        
        # ─── Create row: [Item, Unit, Price1, Price2, ...] ───
        row = [item_name, unit] + prices
        parsed_rows.append(row)
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 5: Create the DataFrame (Table)
    # ─────────────────────────────────────────────────────────────────
    
    # If no data was found, return an empty table
    if len(parsed_rows) == 0:
        return pd.DataFrame(columns=["Item", "Unit"])
    
    # Find the longest row
    max_length = max(len(row) for row in parsed_rows)
    
    # Make all rows the same length by adding None to the end
    for row in parsed_rows:
        while len(row) < max_length:
            row.append(None)
    
    # Create column names: Item, Unit, Col1, Col2, Col3...
    num_price_cols = max_length - 2
    column_names = ["Item", "Unit"] + [f"Col{i}" for i in range(1, num_price_cols + 1)]
    
    # Create the DataFrame
    df = pd.DataFrame(parsed_rows, columns=column_names)
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 6: Clean Up Item Names and Units
    # ─────────────────────────────────────────────────────────────────
    
    # Remove "Rs" from end of item names
    df["Item"] = df["Item"].str.replace(r"Rs$", "", regex=True).str.strip()
    
    # Clean unit column (keep only letters, slash, dot)
    df["Unit"] = df["Unit"].str.replace(r"[^A-Za-z/.]", "", regex=True).str.strip()
    
    # Fix cases where "Rs./kg" ended up in the Item column
    # Example: "Samba Rs./kg" should be split into "Samba" | "Rs./kg"
    
    # Find rows where Unit is empty but Item contains "Rs"
    has_problem = (df["Unit"] == "") | (df["Unit"].isna())
    has_problem = has_problem & df["Item"].str.contains("Rs", na=False)
    
    for index in df[has_problem].index:
        item_text = df.at[index, "Item"]
        
        # Try to split "Samba Rs./kg" into two parts
        match = re.search(r"(.*?)(Rs\.\/[A-Za-z]+)", item_text)
        if match:
            df.at[index, "Item"] = match.group(1).strip()  # "Samba"
            df.at[index, "Unit"] = match.group(2).strip()  # "Rs./kg"
    
    # ─────────────────────────────────────────────────────────────────
    # STEP 7: Convert Prices to Numbers
    # ─────────────────────────────────────────────────────────────────
    
    # Replace text "n.a." with actual None (missing value)
    df = df.replace({"n.a.": None, "N.A.": None})
    
    # Convert all price columns from text to numbers
    for col in df.columns[2:]:  # Skip Item and Unit columns
        df[col] = pd.to_numeric(df[col], errors="coerce")
    
    # ✅ Done! Return the clean table
    return df


# ═══════════════════════════════════════════════════════════════════
# PART 3: Smart Column Names (Detects Locations Automatically)
# ═══════════════════════════════════════════════════════════════════

def create_smart_column_names(section_lines):
    """
    Creates meaningful column names based on what locations are in the data.
    
    Why? The data has prices from different markets and different days.
    Instead of "Col1, Col2, Col3...", we want names like:
        "wholesale_pettah_yesterday", "wholesale_pettah_today", etc.
    
    How it works:
        1. Looks at the first 50 lines to see what locations are mentioned
        2. Checks if "Marandagahamula" market exists
        3. Creates column names for each location and day
    
    Typical structure:
        - Wholesale markets: Pettah, Dambulla (sometimes Marandagahamula)
        - Retail markets: Pettah, Dambulla, Narahenpita
        - Each location has: Yesterday's price + Today's price
    
    Parameters:
        section_lines: List of text lines from the document
    
    Returns:
        List of column names like:
            ["item", "unit", "wholesale_pettah_yesterday", 
             "wholesale_pettah_today", ...]
    """
    
    # ─── Combine first 50 lines and make lowercase ───
    # We only check the beginning because location names appear at the top
    first_50_lines = section_lines[:50]
    combined_text = " ".join(first_50_lines).lower()
    
    # ─── Start with basic columns ───
    column_names = ["item", "unit"]
    
    # ─── Define default locations ───
    # These are the markets where wholesale prices come from
    wholesale_locations = ["pettah", "marandagahamula"]
    
    # These are the markets where retail (customer) prices come from
    retail_locations = ["pettah", "dambulla", "narahenpita"]
    
    # ─── Check if special market exists ───
    # Sometimes "Marandagahamula" market is included
    # If it exists, add it right after Pettah
    # if "marandagahamula" in combined_text:
    #     wholesale_locations.insert(1, "marandagahamula")  # Insert at position 1
    #     # Now list becomes: ["pettah", "marandagahamula", "dambulla"]
    
    # ─── Build wholesale column names ───
    # For each wholesale market, add yesterday and today columns
    for location in wholesale_locations:
        column_names.append(f"wholesale_{location}_yesterday")
        column_names.append(f"wholesale_{location}_today")
    
    # Example result so far:
    # ["item", "unit", "wholesale_pettah_yesterday", "wholesale_pettah_today",
    #  "wholesale_dambulla_yesterday", "wholesale_dambulla_today"]
    
    # ─── Build retail column names ───
    # For each retail market, add yesterday and today columns
    for location in retail_locations:
        column_names.append(f"retail_{location}_yesterday")
        column_names.append(f"retail_{location}_today")
    
    # Final result:
    # ["item", "unit", 
    #  "wholesale_pettah_yesterday", "wholesale_pettah_today",
    #  "wholesale_dambulla_yesterday", "wholesale_dambulla_today",
    #  "retail_pettah_yesterday", "retail_pettah_today",
    #  "retail_dambulla_yesterday", "retail_dambulla_today",
    #  "retail_narahenpita_yesterday", "retail_narahenpita_today"]
    
    return column_names


# ═══════════════════════════════════════════════════════════════════
# PART 4: Extract Section from PDF and Parse It
# ═══════════════════════════════════════════════════════════════════

def extract_and_parse_rice(pdf_path, start_word="RICE", end_word="FISH", page_number=2):
    """
    Extracts the RICE section from a PDF and converts it to a table with smart column names.
    
    This function does three things:
        1. Finds and extracts text between "RICE" and "FISH" in the PDF
        2. Parses that text into a clean DataFrame
        3. Adds meaningful column names based on detected locations
    
    Parameters:
        pdf_path: Path to the PDF file
        start_word: Word that marks the beginning of the section (default: "RICE")
        end_word: Word that marks the end of the section (default: "FISH")
        page_number: Which page to look at (default: 2)
    
    Returns:
        A pandas DataFrame with the parsed price data and smart column names
    """
    
    # NOTE: You need to define extract_section_between() function
    # That function should read the PDF and return lines of text
    section_lines = extract_section_between(pdf_path, start_word, end_word, page_number)
    
    # Parse the extracted lines into a DataFrame
    df = parse_price_section(section_lines)
    
    # ─── Add Smart Column Names ───
    # Generate meaningful column names based on locations
    smart_columns = create_smart_column_names(section_lines)
    
    # Only use as many column names as we have columns in the DataFrame
    # (Sometimes we have fewer columns than expected)
    smart_columns = smart_columns[:len(df.columns)]
    
    # Replace the generic column names (Item, Unit, Col1, Col2...)
    # with meaningful names (item, unit, wholesale_pettah_yesterday...)
    df.columns = smart_columns
    
    return df


# ═══════════════════════════════════════════════════════════════════
# PART 5: Example Usage (How to Run This Code)
# ═══════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Example 1: Test with sample data (no PDF needed)
    # print("=" * 60)
    # print("EXAMPLE 1: Testing with sample data")
    # print("=" * 60)
    
    # sample_lines = [
    #     "Samba Rs./kg 130.00 132.00 135.00 140.00",
    #     "Nadu (White)",
    #     "Rs./kg 125.00 128.00 130.00",
    #     "Red Nadu Rs./kg 120.00 122.00 n.a. 125.00",
    #     "Keeri Samba Rs./kg 140.00 145.00 150.00 155.00 160.00"
    # ]
    
    # result = parse_price_section(sample_lines)
    # print("\n📊 Parsed Table (with generic column names):")
    # print(result)
    
    # # Example 2: Test smart column naming
    # print("\n" + "=" * 60)
    # print("EXAMPLE 2: Testing smart column names")
    # print("=" * 60)
    
    # # Add location info to sample data
    # sample_with_locations = [
    #     "Price Data for Pettah, Dambulla, and Narahenpita markets",
    #     "Samba Rs./kg 130.00 132.00 135.00 140.00 142.00 145.00",
    #     "Nadu (White) Rs./kg 125.00 128.00 130.00 132.00 135.00 138.00"
    # ]
    
    # result2 = parse_price_section(sample_with_locations)
    
    # # Apply smart column names
    # smart_cols = create_smart_column_names(sample_with_locations)
    # smart_cols = smart_cols[:len(result2.columns)]
    # result2.columns = smart_cols
    
    # print("\n📊 Same data with smart column names:")
    # print(result2)
    # print("\n✨ Notice how columns now show: wholesale_pettah_yesterday, etc.")
    
    # Example 3: Parse from actual PDF (uncomment to use)
    print("\n" + "=" * 60)
    print("EXAMPLE 3: Parsing from PDF with smart columns")
    print("=" * 60)
    
    pdf_path = "../data/raw/2024/2024-01-03.pdf"
    df_rice = extract_and_parse_rice(pdf_path)
    
    print("\n✅ DataFrame created successfully!")
    display(df_rice.head(10))
    print("\n📋 Column Names:")
    print(list(df_rice.columns))


EXAMPLE 3: Parsing from PDF with smart columns

✅ DataFrame created successfully!


,item,unit,wholesale_pettah_yesterday,wholesale_pettah_today,wholesale_marandagahamula_yesterday,wholesale_marandagahamula_today,retail_pettah_yesterday,retail_pettah_today,retail_dambulla_yesterday,retail_dambulla_today,retail_narahenpita_yesterday,retail_narahenpita_today
0,Samba,Rs./kg,235.0,235.0,231.0,231.0,250.0,250.0,242.0,242.0,NaN,NaN
1,Nadu,Rs./kg,210.0,210.0,203.0,203.0,220.0,220.0,205.0,214.0,220.0,220.0
2,Kekulu (White),Rs./kg,202.0,202.0,197.0,197.0,210.0,210.0,197.0,192.0,210.0,210.0
3,Kekulu (Red),Rs./kg,180.0,180.0,188.0,187.0,200.0,200.0,192.0,182.0,195.0,195.0
4,Ponni Samba (Imp),Rs./kg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Nadu (Imp),Rs./kgn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Kekulu (White) (Imp),Rs./kgn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN



📋 Column Names:
['item', 'unit', 'wholesale_pettah_yesterday', 'wholesale_pettah_today', 'wholesale_marandagahamula_yesterday', 'wholesale_marandagahamula_today', 'retail_pettah_yesterday', 'retail_pettah_today', 'retail_dambulla_yesterday', 'retail_dambulla_today', 'retail_narahenpita_yesterday', 'retail_narahenpita_today']
